In [1]:
numeric_stats = ['derived_fg', 'derived_ft', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
fantasy_stats = ['d_FG%', 'd_FT%', 'FG%', 'FT%','TRB', 'AST', 'STL', 'BLK', 'TOV', '3P', 'PTS']
negative_stats = ['FTA', 'TOV', 'FGA']

In [2]:
import unidecode

def name_transform(name):
    name = name.split('\\')[0].strip()
    name = unidecode.unidecode(name)
    transforms = {}
    transforms['Marcus Morris'] = 'Marcus Morris Sr.'
    transforms['Otto Porter'] = 'Otto Porter Jr.'
    if name in transforms:
        return transforms[name]
    return name
name_transform('Otto Porter')

'Otto Porter Jr.'

In [3]:
# Import stats CSV
import pandas as pd

arr = pd.read_csv('current_season.csv')
arr['Player'] = arr['Player'].transform(name_transform)
arr = arr.drop(['Rk', 'GS', 'Age', 'Pos', 'FG%', 'FT%', '2P%', 'eFG%', '3PA', '3P%', '2P', '2PA', 'Tm'], axis=1)
arr = arr.groupby(arr['Player']).sum()
arr['FT%'] = arr['FT'] * 100 / arr['FTA']
arr['FG%'] = arr['FG'] * 100 / arr['FGA']
arr['d_FG%'] = arr['FG'] * (arr['FG%'] - arr['FG%'].mean())
arr['d_FT%'] = arr['FT'] * (arr['FT%'] - arr['FT%'].mean())
# arr['TOV'] = -1 * arr['TOV']
arr[['TRB', 'AST', 'STL', 'BLK', 'TOV', '3P', 'PTS']] = arr[['TRB', 'AST', 'STL', 'BLK', 'TOV', '3P', 'PTS']].div(arr['G'], axis=0)

In [4]:
# Import average draft values TSV
raw_draft_values = pd.read_csv('draft_values.tsv', sep='\t')
raw_draft_values
def transform_name(name):
    notes_loc = name.find('Notes')
    if notes_loc > -1:
        name = name[notes_loc+5:]
    note_loc = name.find('Note')
    if note_loc > -1:
        name = name[note_loc+4:]
    names = name.split()
    ret = names[0:2]
    if (names[2] == 'Jr.' or names[2] == 'Sr.'):
        ret = names[0:3]
    return ' '.join(ret)

def transform_average(val):
    val = val[1:]
    try:
        num_val = float(val)
        return num_val
    except ValueError:
        return 0

raw_draft_values['Name'] = raw_draft_values['Name'].transform(transform_name)
raw_draft_values['Avg $'] = raw_draft_values['Average'].transform(transform_average)
raw_draft_values = raw_draft_values.set_index('Name')
arr = arr.join(raw_draft_values[['Avg $']])
arr = arr.fillna(0)

In [5]:
arr['TOV'] = -1 * arr['TOV']
print(arr[fantasy_stats+['Avg $']].corr().to_json())
arr['TOV'] = -1 * arr['TOV']

{"d_FG%":{"d_FG%":1.0,"d_FT%":0.0589357518,"FG%":0.5047997241,"FT%":0.0830901387,"TRB":0.5605958131,"AST":0.1698993365,"STL":0.1939384774,"BLK":0.413440956,"TOV":-0.3163890891,"3P":0.0144757899,"PTS":0.4414098998,"Avg $":0.4061963887},"d_FT%":{"d_FG%":0.0589357518,"d_FT%":1.0,"FG%":0.0062236643,"FT%":0.3621649847,"TRB":0.0582099769,"AST":0.4437849693,"STL":0.2280451849,"BLK":-0.0508608097,"TOV":-0.3565445404,"3P":0.4720935452,"PTS":0.4889314301,"Avg $":0.4469779533},"FG%":{"d_FG%":0.5047997241,"d_FT%":0.0062236643,"FG%":1.0,"FT%":0.1983534852,"TRB":0.3895391922,"AST":0.122398265,"STL":0.1894961766,"BLK":0.2801778839,"TOV":-0.1346448029,"3P":0.1006730927,"PTS":0.317290725,"Avg $":0.1551694072},"FT%":{"d_FG%":0.0830901387,"d_FT%":0.3621649847,"FG%":0.1983534852,"FT%":1.0,"TRB":0.2387477435,"AST":0.3022742753,"STL":0.34803246,"BLK":0.0601591911,"TOV":-0.2619783197,"3P":0.3994671236,"PTS":0.4201834491,"Avg $":0.1862462748},"TRB":{"d_FG%":0.5605958131,"d_FT%":0.0582099769,"FG%":0.3895391922

In [6]:
arr['Name'] = arr.index
print(arr[fantasy_stats+['Name','Avg $']].to_json(orient='records'))

[{"d_FG%":103.7227804878,"d_FT%":-565.074441123,"FG%":44.8087431694,"FT%":60.8108108108,"TRB":7.5,"AST":4.375,"STL":0.75,"BLK":1.0,"TOV":2.625,"3P":1.6875,"PTS":14.75,"Name":"Aaron Gordon","Avg $":5.7},{"d_FG%":-329.0433274365,"d_FT%":49.8265635989,"FG%":36.0655737705,"FT%":83.3333333333,"TRB":1.2941176471,"AST":2.1764705882,"STL":0.7058823529,"BLK":0.1764705882,"TOV":0.6470588235,"3P":1.0,"PTS":6.4705882353,"Name":"Aaron Holiday","Avg $":0.0},{"d_FG%":-98.9965627821,"d_FT%":0.0,"FG%":35.2941176471,"FT%":0.0,"TRB":1.8571428571,"AST":0.5714285714,"STL":0.0,"BLK":0.2857142857,"TOV":0.5714285714,"3P":1.2857142857,"PTS":4.7142857143,"Name":"Aaron Nesmith","Avg $":0.0},{"d_FG%":28.1199385374,"d_FT%":-26.8054157875,"FG%":47.0588235294,"FT%":66.6666666667,"TRB":2.5,"AST":1.0,"STL":0.25,"BLK":0.0,"TOV":0.25,"3P":0.5,"PTS":5.5,"Name":"Abdel Nader","Avg $":0.0},{"d_FG%":-39.1477274897,"d_FT%":0.0,"FG%":35.7142857143,"FT%":0.0,"TRB":0.6666666667,"AST":0.6666666667,"STL":0.3333333333,"BLK":0.16666